## Instalação da biblioteca holidays (feriados) e import de todas as bibliotecas utilizadas no código

In [1]:

#!pip install holidays #--index-url http://artifactory.santanderbr.corp/artifactory/api/pypi/pypi-all/simple --trusted-host artifactory.santanderbr.corp
#!pip install holidays
import pandas as pd
import requests, json, holidays
from pyspark.sql.functions import to_timestamp, get_json_object, current_timestamp
from pyspark.sql.types import IntegerType
from pyspark.sql import functions as F, SparkSession
from datetime import datetime, timedelta
from delta import configure_spark_with_delta_pip
from funcoes import dados_sensiveis

In [2]:
builder = SparkSession.builder \
    .appName("Table Validator") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.sql.warehouse.dir", "./spark-warehouse") \
    .config("javax.jdo.option.ConnectionURL", "jdbc:derby:./metastore_db;create=true") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()


25/06/04 23:31:27 WARN Utils: Your hostname, adones-Nitro5 resolves to a loopback address: 127.0.1.1; using 192.168.15.35 instead (on interface wlp0s20f3)
25/06/04 23:31:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/adones/Documentos/Python_Projects/table_validator/validator/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/adones/.ivy2/cache
The jars for the packages stored in: /home/adones/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a5277702-80e0-48c0-b6c8-2e3150e19c81;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.2 in central
	found io.delta#delta-storage;3.3.2 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 210ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.2 from central in [default]
	io.delta#delta-storage;3.3.2 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0

## Criando as variáveis de data

In [3]:

dt_hj = datetime.today()
d_sem = dt_hj.weekday()
dt_hj_str = datetime.today().strftime("%Y-%m-%d")
exec_datetime = datetime.now()
exec_timestamp_str = exec_datetime.strftime('%Y-%m-%d %H:%M:%S')
ano_mes_corr = exec_datetime.strftime('%Y-%m')
dia_20 = f"{ano_mes_corr}-20"
dt_inic_mes = f"{ano_mes_corr}-01"

## Função para obter qual o dia útil do mês estamos

In [4]:
feriados_br = holidays.Brazil()

def business_days(start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    num_business_days = 0
    current_date = start_date

    while current_date <= end_date:
        if current_date.weekday() < 5 and current_date not in feriados_br:
            num_business_days += 1
        current_date += timedelta(days=1)

    if end_date.weekday() < 5 and end_date in feriados_br:
        num_business_days += 1

    return num_business_days

dia_util = business_days(dt_inic_mes, dt_hj_str)


## Obtendo todas as tabelas automaticamente pelos schemas 
(Isso faz com que você faça a validação de todas as tabelas disponiveis nos schemas)

In [5]:
lista_schemas = ["s_analytics_db", "s_sales_data_db", "s_financial_risk_db", "s_customer_db"] #Coloque os schemas das tabelas que deseja validar

df_schemas = None
for i in lista_schemas:
    df_sch = spark.sql(f"SHOW TABLES IN {i}")
    if df_schemas is None:
        df_schemas = df_sch
    else:
        df_schemas = df_schemas.union(df_sch).dropDuplicates(["tableName"])
df_schemas.show(truncate=False)

25/06/04 23:31:37 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/06/04 23:31:37 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
25/06/04 23:31:40 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
25/06/04 23:31:40 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore adones@127.0.1.1
25/06/04 23:31:40 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


+-------------------+------------------------+-----------+
|namespace          |tableName               |isTemporary|
+-------------------+------------------------+-----------+
|s_customer_db      |abandoned_carts         |false      |
|s_analytics_db     |ad_clicks               |false      |
|s_analytics_db     |conversion_rates        |false      |
|s_financial_risk_db|credit_exposure         |false      |
|s_sales_data_db    |customer_returns        |false      |
|s_customer_db      |feedback_surveys        |false      |
|s_financial_risk_db|interest_rate_scenarios |false      |
|s_analytics_db     |marketing_campaigns     |false      |
|s_customer_db      |newsletter_subscriptions|false      |
|s_sales_data_db    |order_items             |false      |
|s_sales_data_db    |orders                  |false      |
|s_sales_data_db    |payment_receipts        |false      |
|s_customer_db      |purchase_funnels        |false      |
|s_financial_risk_db|risk_limits             |false     

### Selecionando apenas as tabelas que precisam ser verificadas no dia
Aqui conseguimos retirar da validação do dia aquelas tabelas que precisam ser validadas apenas em dias específicosx por exemplo tabelas que são populadas apenas uma vez por mês ou uma vez por semana, ou tabelas que não são populadas no final de semana etc.

In [7]:
df_tabelas = df_schemas \
    .withColumn("nome_tabela", 
                F.concat(
                    F.col("namespace"), 
                    F.lit("."), 
                    F.col("tableName"))) \
    .withColumn("exec_hj", #Criamos uma coluna para saber as tabelas que devem ser populadas hoje, a coluna terá apenas "SIM" ou "NÃO" para cada tabela
        F.when(
            F.col("nome_tabela").isin("s_customer_db.abandoned_carts ", "s_customer_db.abandoned_carts") & #coloque aqui as tabelas que só rodam de sábado e domingo
            (~F.lit(d_sem).isin(5, 6)), F.lit("Não")
        ).when(
            F.col("nome_tabela").isin("s_sales_data_db.invoices", "s_analytics_db.marketing_campaigns") & #coloque aqui as tabelas que só rodam de sexta-feira
            (~F.lit(d_sem).isin(4)), F.lit("Não")
        ).when(
            F.col("nome_tabela").isin("s_financial_risk_db.market_positions") & #coloque aqui as tabelas que só rodam todo dia 20 de cada mês
            (F.lit(dt_hj_str) != F.lit(dia_20)), F.lit("Não") \
        ).when(
            F.col("nome_tabela").isin("s_financial_risk_db.risk_limits") & #coloque aqui as tabelas que NÃO rodam de sábado e domingo
            (F.lit(d_sem).isin(5, 6)), F.lit("Não") \
        ).when(
            F.col("nome_tabela").isin("s_financial_risk_db.market_positions") & #tabelas que só rodam no 4º dia útil
            (~F.lit(dia_util).isin(4)), F.lit("Não")
        #Aqui você pode configurar da forma que precisar, coloquei alguns exemplos porém fica à critério de cada um
        ).otherwise(F.lit("Sim"))) \
    .filter(
        (~F.col("nome_tabela").isin("s_customer_db.user_sessions", "s_financial_risk_db.liquidity_reports")) & #tabelas que não quer fazer a validação por algum motivo, coloque aqui para retirar do validador
        (F.col("exec_hj").isin("Sim"))) \
    .select("nome_tabela") #fazemos o filtro acima para no final o dataframe ficar apenas com o nome das tabelas que precisam ser validadas no dia da execução
    
df_tabelas.show(truncate=False)

+-------------------------------------------+
|nome_tabela                                |
+-------------------------------------------+
|s_analytics_db.ad_clicks                   |
|s_analytics_db.conversion_rates            |
|s_financial_risk_db.credit_exposure        |
|s_sales_data_db.customer_returns           |
|s_customer_db.feedback_surveys             |
|s_financial_risk_db.interest_rate_scenarios|
|s_customer_db.newsletter_subscriptions     |
|s_sales_data_db.order_items                |
|s_sales_data_db.orders                     |
|s_sales_data_db.payment_receipts           |
|s_customer_db.purchase_funnels             |
|s_financial_risk_db.risk_limits            |
|s_sales_data_db.sales_targets              |
|s_analytics_db.session_durations           |
|s_financial_risk_db.stress_test_results    |
|s_analytics_db.website_traffic_logs        |
+-------------------------------------------+



### Aqui é feito um teste para ver se as tabelas tem permissão de acesso.
Se tiver alguma tabela que não possui acesso ela será retirada do validador para não quebrar o código e será colocada em uma lista de tabelas sem permissão e no final vai na mensagem informando que não tem acesso

In [8]:
tab_sem_per = []
for tb_name in df_tabelas.collect():
    try:
        spark.sql(f"select * from {tb_name['nome_tabela']} limit 1")
    except:
        tab_sem_per.append(tb_name['nome_tabela'])
df_tabelas = df_tabelas.filter(~F.col("nome_tabela").isin(tab_sem_per))
print(tab_sem_per)

[]


### Verificando se o job executou ou não
Aqui coletamos os dados da tabela através do "describe history" pegando a ultima atualização e trazendo dados como data e horário da execução, quantidade de linhas inseridas, nome da tabela e do job.
Aqui é feito um loop no dataframe de tabelas para pegar os dados de todas as tabelas que estão para ser validadas no dia da execução

In [9]:
df_final = None
for table_name in df_tabelas.collect():
    history_df = spark.sql(f"""DESCRIBE HISTORY {table_name['nome_tabela']}""") \
        .filter( #Pegando apenas escrita de tabela e desconsiderando outras operações
            (~F.col("operation").isin("VACUUM START", "VACUUM END", "MERGE"))) \
        .withColumn("data_exec", 
                    F.to_date(F.col("timestamp"), "yyyy-MM-dd HH:mm:ss")) \
        .withColumn("hr_exec", 
                    F.date_format(
                        (F.to_timestamp("timestamp", "yyyy-MM-dd HH:mm:ss") 
                        - F.expr("INTERVAL 3 HOURS")), "HH:mm:ss")) \
        .withColumn("nome_tabela_Desc", 
                    F.lit(table_name["nome_tabela"]))

    # Obtendo apenas a última execução de cada tabela
    recent_update_df = history_df.orderBy(F.col("timestamp").desc()).limit(1)

    recent_update_df = recent_update_df \
        .withColumn("operationMetricsStr", 
                    F.to_json(F.col("operationMetrics"))) \
        .withColumn("job", F.to_json(F.col("job")))

    #obtendo o nome do job e a quantidade de linhas inseridas
    recent_update_df = recent_update_df \
        .withColumn("qtd_linhas", 
                    F.get_json_object(F.col("operationMetricsStr"), "$.numOutputRows").cast(IntegerType())) \
        .withColumn("job_name",
                    F.get_json_object(F.col("job"), "$.jobName"))

    recent_update_df = recent_update_df \
        .select(
            "nome_tabela_Desc",
            "qtd_linhas",
            "data_exec",
            "hr_exec",            
            "job_name"
        )

    if df_final is None: # Concatenando os dados no DataFrame final
        df_final = recent_update_df
    else:
        df_final = df_final.union(recent_update_df)

df_final.show(truncate=False)

+-------------------------------------------+----------+----------+--------+--------+
|nome_tabela_Desc                           |qtd_linhas|data_exec |hr_exec |job_name|
+-------------------------------------------+----------+----------+--------+--------+
|s_analytics_db.ad_clicks                   |194       |2025-06-04|20:30:21|NULL    |
|s_analytics_db.conversion_rates            |186       |2025-06-04|20:30:23|NULL    |
|s_financial_risk_db.credit_exposure        |178       |2025-06-04|20:30:25|NULL    |
|s_sales_data_db.customer_returns           |170       |2025-06-04|20:30:27|NULL    |
|s_customer_db.feedback_surveys             |162       |2025-06-04|20:30:29|NULL    |
|s_financial_risk_db.interest_rate_scenarios|154       |2025-06-04|20:30:30|NULL    |
|s_customer_db.newsletter_subscriptions     |138       |2025-06-04|20:30:34|NULL    |
|s_sales_data_db.order_items                |130       |2025-06-04|20:30:35|NULL    |
|s_sales_data_db.orders                     |122      

### Aplicando status final
Aqui é criada a coluna "status" que vai mostrar se o job deu "Sucesso", "Falha" ou "Executou hoje porém não escreveu dado"

In [13]:
join_final = df_tabelas.alias("a") \
    .join(df_final.alias("b"), 
          on=F.col("a.nome_tabela") == F.col("b.nome_tabela_Desc"), 
          how="left") \
    .withColumn("status", 
        F.when((F.col("qtd_linhas") > 0) & (F.col("data_exec") == dt_hj_str), "Sucesso") \
        .otherwise(F.lit("Falha"))) \
    .withColumn("status", 
        F.when(
            (F.col("qtd_linhas").isNull()) | (F.col("qtd_linhas") == 0) & 
            (F.col("data_exec") == dt_hj_str), "Executou hoje porém não escreveu dado") \
        .otherwise(F.col("status"))) \
    .withColumn("DT_REFE",
        F.lit(dt_hj_str)) \
    .select(
        "nome_tabela",
        "job_name",
        "qtd_linhas",
        "data_exec",
        "hr_exec",      
        "status",
        "DT_REFE"
    )

join_final.show(truncate=False)

+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|nome_tabela                                |job_name|qtd_linhas|data_exec |hr_exec |status |DT_REFE   |
+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|s_analytics_db.ad_clicks                   |NULL    |194       |2025-06-04|20:30:21|Sucesso|2025-06-04|
|s_analytics_db.conversion_rates            |NULL    |186       |2025-06-04|20:30:23|Sucesso|2025-06-04|
|s_financial_risk_db.credit_exposure        |NULL    |178       |2025-06-04|20:30:25|Sucesso|2025-06-04|
|s_sales_data_db.customer_returns           |NULL    |170       |2025-06-04|20:30:27|Sucesso|2025-06-04|
|s_customer_db.feedback_surveys             |NULL    |162       |2025-06-04|20:30:29|Sucesso|2025-06-04|
|s_financial_risk_db.interest_rate_scenarios|NULL    |154       |2025-06-04|20:30:30|Sucesso|2025-06-04|
|s_customer_db.newsletter_subscriptions     |NULL    |1

### Obtendo o horário atual e filtrando o dataframe final
Como o job roda de 1 em 1 hora, fiz essa validação para ele pegar apenas os job's que rodam da hora que está sendo executado para baixo para não ter perigo de aplicar "Falha" a um job que só roda algum tempo depois do horário que está sendo feita a validação

In [11]:
current_hour = str(spark.sql("SELECT hour(current_timestamp()) as current_hour").collect()[0][0] - 3).zfill(2)
hr_1 = f"{current_hour}:00:00"
join_final = join_final.filter(
    F.to_timestamp("hr_exec", "HH:mm:ss") <= F.to_timestamp(F.lit(hr_1), "HH:mm:ss"))

### Criando o dataframe de falhas
Aqui filtramos o dataframe final para ter apenas "Falha" ou "Executou hoje porém não escreveu dado"

In [14]:
df_falhas = join_final#.filter((F.col("status") == "Falha") | (F.col("status") == "Executou hoje porém não escreveu dado"))
df_falhas.show(truncate=False)

+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|nome_tabela                                |job_name|qtd_linhas|data_exec |hr_exec |status |DT_REFE   |
+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|s_analytics_db.ad_clicks                   |NULL    |194       |2025-06-04|20:30:21|Sucesso|2025-06-04|
|s_analytics_db.conversion_rates            |NULL    |186       |2025-06-04|20:30:23|Sucesso|2025-06-04|
|s_financial_risk_db.credit_exposure        |NULL    |178       |2025-06-04|20:30:25|Sucesso|2025-06-04|
|s_sales_data_db.customer_returns           |NULL    |170       |2025-06-04|20:30:27|Sucesso|2025-06-04|
|s_customer_db.feedback_surveys             |NULL    |162       |2025-06-04|20:30:29|Sucesso|2025-06-04|
|s_financial_risk_db.interest_rate_scenarios|NULL    |154       |2025-06-04|20:30:30|Sucesso|2025-06-04|
|s_customer_db.newsletter_subscriptions     |NULL    |1

### Envio da notificação pelo Teams
Aqui é feita a verificação se o df_falhas tiver alguma linha enviamos a notificação para o Teams com o dataframe de falhas para que possa ser feita uma ação o quanto antes, caso contrário a notificação informará que não teve falha em job's no horário da verificação

In [15]:
if df_falhas.count() >= 1:
    webhook_url = dados_sensiveis.wb_str()
    tb_dash_html = df_falhas.toPandas().to_html(index=False) #transformando o dataframe pyspark em html pra enviar via teams
    if len(tab_sem_per) > 0:
        message = {
            "title": f"FALHA DE JOB DAS {current_hour}:00 - Timestamp validado: {exec_timestamp_str}",
            "text": f"ATENÇÃO! ALGUM JOB EXECUTOU ATÉ AS (current_hour):59:00 E EXECUTOU COM FALHA OU NÃO ESCREVEU DADO NA DATA DE HOJE ({dt_hj_str})***<br>"
                    f"{tb_dash_html}<br>"
                    f"Essas tabelas sem permissão de acesso conforme abaixo:<br>"
                    f"{tab_sem_per}<br>"
                    f"FAVOR PEDIR PARA LIBERAR O ACESSO AS TABELAS<br>"
        }
    else:
        message = {
            "title": f"FALHA DE JOB - Timestamp validado: {exec_timestamp_str}",
            "text": f"ATENÇÃO! ALGUM JOB EXECUTOU ATÉ AS {current_hour}:59:00 E EXECUTOU COM FALHA OU NÃO ESCREVEU DADO NA DATA DE HOJE ({dt_hj_str})***<br>"
                    f"{tb_dash_html}<br>"
        }

    #proxies = {"http": "coloque aqui seu proxy", "https": "coloque aqui seu proxy"}

    try:
        response = requests.post(
            webhook_url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(message),
            timeout=30,
            verify=False,
            #proxies=proxies
        )

        if response.status_code == 200:
            print("Mensagem enviada com sucesso!")
        else:
            print(f"Falha ao enviar mensagem. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao enviar mensagem: {e}")

else:
    webhook_url = dados_sensiveis.wb_str()

    tb_dash_html = df_falhas.toPandas().to_html(index=False)
    if len(tab_sem_per) > 0:
        message = {
            "title": f"VALIDAÇÃO DE JOB'S DAS {current_hour}:00 - Timestamp validado: {exec_timestamp_str}",
            "text": f"*****ATENÇÃO TODOS OS JOB'S QUE RODAM ATÉ AS {current_hour}:59:00 EXECUTARAM COM SUCESSO!!!*****<br>"
                    f"<br>Existem tabelas sem permissão de acesso conforme abaixo:<br>"
                    f"{tab_sem_per}<br>"
                    f"<br>FAVOR PEDIR PARA LIBERAR O ACESSO AS TABELAS<br>*****"
        }
    else:
        message = {
            "title": f"VALIDAÇÃO DE JOB'S DAS {current_hour}:00 - Timestamp validado: {exec_timestamp_str}",
            "text": f"*****ATENÇÃO TODOS OS JOB'S QUE RODAM ATÉ AS {current_hour}:59:00 EXECUTARAM COM SUCESSO!!!*****<br>"
        }

    #proxies = {"http": "coloque aqui seu proxy", "https": "coloque aqui seu proxy"}

    try:
        response = requests.post(
            webhook_url,
            headers={"Content-Type": "application/json"},
            data=json.dumps(message),
            timeout=30,
            verify=False,
            #p#oxies=proxies
        )

        if response.status_code == 200:
            print("Mensagem enviada com sucesso!")
        else:
            print(f"Falha ao enviar mensagem. Status code: {response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Erro ao enviar mensagem: {e}")

/home/adones/Documentos/Python_Projects/table_validator/validator/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'alunoimpacta.webhook.office.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Mensagem enviada com sucesso!


### Salvando o log na tabela de validação para caso necessite de histórico

In [18]:
#colocar aqui para popular a tabela de log

join_final.write.format("delta") \
    .mode("append") \
    .partitionBy("DT_REFE") \
    .saveAsTable("sandbox.validador_jobs")

25/06/04 23:37:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/06/04 23:37:02 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`sandbox`.`validador_jobs` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/06/04 23:37:02 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
25/06/04 23:37:03 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
25/06/04 23:37:03 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
25/06/04 23:37:03 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [23]:
spark.sql("select * from sandbox.validador_jobs").show(truncate=False)

+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|nome_tabela                                |job_name|qtd_linhas|data_exec |hr_exec |status |DT_REFE   |
+-------------------------------------------+--------+----------+----------+--------+-------+----------+
|s_analytics_db.ad_clicks                   |NULL    |194       |2025-06-04|20:30:21|Sucesso|2025-06-04|
|s_analytics_db.conversion_rates            |NULL    |186       |2025-06-04|20:30:23|Sucesso|2025-06-04|
|s_financial_risk_db.credit_exposure        |NULL    |178       |2025-06-04|20:30:25|Sucesso|2025-06-04|
|s_sales_data_db.customer_returns           |NULL    |170       |2025-06-04|20:30:27|Sucesso|2025-06-04|
|s_customer_db.feedback_surveys             |NULL    |162       |2025-06-04|20:30:29|Sucesso|2025-06-04|
|s_financial_risk_db.interest_rate_scenarios|NULL    |154       |2025-06-04|20:30:30|Sucesso|2025-06-04|
|s_customer_db.newsletter_subscriptions     |NULL    |1